In [ ]:
import pandas as pd
from google.cloud import firestore
import os

# --- Project Setup ---
project_id = 'salrow-aa320'
os.environ['GCLOUD_PROJECT'] = project_id

# --- Firestore Connection ---
try:
    db = firestore.Client(project=project_id)
    print(f"Successfully connected to Firestore project: {project_id}")
    accidents_ref = db.collection('accidents')
    print("Successfully created a reference to the 'accidents' collection.")
except Exception as e:
    print(f"Error connecting to Firestore: {e}")
    print("Please ensure your service account key is correctly configured and has permissions.")

# --- Data Fetching ---
print('\nFetching all documents from the \'accidents\' collection...')
# This will now fetch documents with all 27 columns
docs = [doc.to_dict() for doc in accidents_ref.stream()]

if not docs:
    print('No documents found in the collection. Please ensure you have run the upload script.')
else:
    print(f"Successfully fetched {len(docs)} documents.")
    df = pd.DataFrame(docs)

    # --- Initial Data Analysis ---
    print("\n--- Data Analysis ---")
    print("DataFrame Info (with new columns):")
    # Displaying info will show us all the new columns and their data types
    df.info()

    print("\nFirst 5 rows of the new DataFrame:")
    # Displaying the head will give a preview of the actual data
    pd.set_option('display.max_columns', None) # Ensure all columns are displayed
    print(df.head())


In [ ]:
# --- Step 2: Feature Engineering - Calculate Accident Duration ---

# Ensure the time columns are in the correct datetime format
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['End_Time'] = pd.to_datetime(df['End_Time'])

# Calculate the duration in minutes and create a new 'Duration' column
# We get the total seconds of the time difference and convert it to minutes
df['Duration'] = (df['End_Time'] - df['Start_Time']).dt.total_seconds() / 60

# --- Handle Potential Data Issues ---
# A small number of records might have a zero or negative duration due to data errors.
# For our analysis, we will filter these out to avoid skewing the results.
initial_count = len(df)
df = df[df['Duration'] > 0].copy()
print(f"Removed {initial_count - len(df)} records with invalid (zero or negative) durations.")
print(f"Now using {len(df)} valid records for analysis.")

# --- Step 3: Exploratory Analysis - The Impact of Junctions ---

print("\n--- Analysis: Average Accident Duration by Junction ---")

# Group the data by the 'Junction' column and calculate the average duration for each case
junction_impact = df.groupby('Junction')['Duration'].mean()

# Print the results in a clear, readable format
print(f"Average duration for accidents WITHOUT a junction: {junction_impact[False]:.2f} minutes")
print(f"Average duration for accidents WITH a junction:    {junction_impact[True]:.2f} minutes")

# Show the first few rows with our new 'Duration' column
print("\n--- DataFrame Preview ---")
print(df[['ID', 'Start_Time', 'End_Time', 'Duration', 'Junction']].head())


In [ ]:
# --- Step 3 (Corrected): Comprehensive Analysis of All Road Conditions ---

# List of all the boolean road condition columns we want to analyze
# CORRECTED: 'Give_Way' is now included in the list.
road_condition_columns = [
    'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
    'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
    'Turning_Loop'
]

print("--- Analysis: Impact of All Road Conditions on Average Accident Duration ---")

# Loop through each road condition column and perform the same analysis as before
for column in road_condition_columns:
    print(f"\n----- Impact of: {column} -----")

    # Group by the current road condition and calculate the average duration
    impact_analysis = df.groupby(column)['Duration'].mean()

    # Use .get() to safely access the results, preventing errors if a
    # category (e.g., True) doesn't exist for a particular feature.
    avg_duration_false = impact_analysis.get(False, 0)
    avg_duration_true = impact_analysis.get(True, 0)

    # Print the results
    if avg_duration_false > 0:
        print(f"Average duration WITHOUT this feature: {avg_duration_false:.2f} minutes")
    else:
        print("No accidents recorded without this feature.")

    if avg_duration_true > 0:
        print(f"Average duration WITH this feature:    {avg_duration_true:.2f} minutes")
    else:
        print("No accidents recorded with this feature.")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# --- Step 4: Prepare Data for Modeling (No changes) ---

feature_columns = [
    'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
    'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
    'Turning_Loop'
]
target_column = 'Duration'

X = df[feature_columns]
y = df[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Step 5: Train and Evaluate Model (No changes) ---

dt_model = DecisionTreeRegressor(max_depth=5, random_state=42)
dt_model.fit(X_train, y_train)
y_pred = dt_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("\n--- Model Performance (using RMSE) ---")
print(f"The model's prediction error (Root Mean Squared Error) is: {rmse:.2f} minutes.")


# --- CORRECTED: Interpret the Results with a Clean Index ---

# Get the importance of each feature in making predictions
feature_importances = pd.DataFrame({
    'feature': feature_columns,
    'importance': dt_model.feature_importances_
}).sort_values('importance', ascending=False).reset_index(drop=True) # <-- This fixes the index

print("\n--- Most Important Predictors of Accident Duration (Corrected Sort) ---")
print(feature_importances)

In [ ]:
# --- Corrected Analysis: Focusing on the Top 5 Most Important Predictors ---

print("--- Analysis Part 1: Average Duration by Severity Level (Overall) ---")

# This part remains useful for context
severity_impact = df.groupby('Severity')['Duration'].mean().sort_index()
for severity_level, avg_duration in severity_impact.items():
    print(f"Severity Level {int(severity_level)}: Average accident duration is {avg_duration:.2f} minutes.")

print("\n" + "="*50 + "\n")

print("--- Analysis Part 2: Detailed Breakdown for the TOP 5 PREDICTORS ---")

# CORRECTED: Define a list with only the top 5 most important features
top_5_predictors = [
    'Junction',
    'Station',
    'Crossing',
    'Traffic_Signal',
    'Stop'
]

# Loop through only the top predictors to analyze their impact within each severity level
for column in top_5_predictors:
    print(f"\n----- Impact of: {column} (broken down by Severity) -----")

    # Group by both Severity and the current road condition column
    # Use .unstack() to pivot the data into a readable table format
    detailed_impact = df.groupby(['Severity', column])['Duration'].mean().unstack()

    # Rename columns for clarity
    detailed_impact.rename(columns={False: f'WITHOUT Feature', True: f'WITH Feature'}, inplace=True)

    # Display the formatted results
    print(detailed_impact.to_string(float_format="%.2f min", na_rep="--"))

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# --- Part 2: Predictive Analysis with Severity ---
print("Building the new, more powerful predictive model including 'Severity'.")

# --- Step 1: Prepare the Data for the New Model ---

# Our features now include the top 5 road conditions AND the 'Severity' level.
top_5_predictors = [
    'Junction',
    'Station',
    'Crossing',
    'Traffic_Signal',
    'Stop'
]
features_with_severity = top_5_predictors + ['Severity']

# Create a copy of the DataFrame to work with
model_df = df[features_with_severity + ['Duration']].copy()

# **Crucial Step: One-Hot Encode the 'Severity' column**
# We use pd.get_dummies() to convert the single 'Severity' column (with values 1, 2, 3, 4)
# into four new columns: 'Severity_1', 'Severity_2', etc. This treats each level as a distinct category.
model_df = pd.get_dummies(model_df, columns=['Severity'], prefix='Severity')

print("\n--- Data Preparation ---")
print("The 'Severity' column has been converted into separate features:")
print(model_df.head())

# --- Step 2: Define New Features (X) and Target (y) ---

# The target remains the same
target_column = 'Duration'

# The features are all columns in our new DataFrame except for the target
X_new = model_df.drop(target_column, axis=1)
y_new = model_df[target_column]

# Get the final list of feature columns for our report
final_feature_columns = X_new.columns.tolist()

# Split the data into training and testing sets
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

# --- Step 3: Train and Evaluate the New Model ---

# Initialize and train a new Decision Tree model
dt_model_new = DecisionTreeRegressor(max_depth=5, random_state=42)
dt_model_new.fit(X_train_new, y_train_new)

# Make predictions and evaluate the new model using RMSE
y_pred_new = dt_model_new.predict(X_test_new)
mse_new = mean_squared_error(y_test_new, y_pred_new)
rmse_new = np.sqrt(mse_new)

print("\n--- New Model Performance ---")
print(f"The new model's prediction error (RMSE) is: {rmse_new:.2f} minutes.")
print("This should be a significant improvement over the previous model which did not include 'Severity'.")


# --- Step 4: Find Most Important Predictors in the New Model ---

# Get the importance of each feature in the new model
feature_importances_new = pd.DataFrame({
    'feature': final_feature_columns,
    'importance': dt_model_new.feature_importances_
}).sort_values('importance', ascending=False).reset_index(drop=True)

print("\n--- Most Important Predictors (Model with Severity) ---")
print(feature_importances_new)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# --- Visualize Model Performance: Actual vs. Predicted (with Zoom) ---

# --- Step 1: Calculate the Zoom Limits ---
# To zoom in, we'll find the 95th percentile of the actual and predicted durations.
# This helps us ignore the extreme outliers that squash the plot.
limit_value = max(np.percentile(y_test_new, 95), np.percentile(y_pred_new, 95))
# We add a 10% buffer for a nice visual margin.
zoom_limit = limit_value * 1.1

# Calculate residuals for the second plot
residuals = y_test_new - y_pred_new
# We'll also zoom the residual plot's y-axis to see the error distribution more clearly.
residual_zoom_limit = np.percentile(np.abs(residuals), 95) * 1.2 # 95th percentile of absolute error, with 20% buffer

# --- Step 2: Create the Plots ---
# Create a figure with two subplots (side-by-side)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# --- Plot 1: Actual vs. Predicted Duration ---
ax1.scatter(y_test_new, y_pred_new, alpha=0.5, label='Model Predictions')
ax1.plot([0, zoom_limit], [0, zoom_limit], 'r--', lw=2, label='Perfect Prediction') # Adjust line to new zoom
ax1.set_xlabel('Actual Accident Duration (minutes)', fontsize=12)
ax1.set_ylabel('Predicted Accident Duration (minutes)', fontsize=12)
ax1.set_title('Decision Tree: Actual vs. Predicted (Zoomed In)', fontsize=16)
ax1.grid(True)
# ** NEW: Set the axis limits to zoom in **
ax1.set_xlim(0, zoom_limit)
ax1.set_ylim(0, zoom_limit)
ax1.legend()


# --- Plot 2: Residual Plot ---
ax2.scatter(y_pred_new, residuals, alpha=0.5, edgecolors='g')
ax2.axhline(y=0, color='r', linestyle='--', lw=2)
ax2.set_xlabel('Predicted Accident Duration (minutes)', fontsize=12)
ax2.set_ylabel('Residual (Error) in minutes', fontsize=12)
ax2.set_title('Residual Plot (Zoomed In)', fontsize=16)
ax2.grid(True)
# ** NEW: Set the axis limits to zoom in **
ax2.set_xlim(0, zoom_limit)
ax2.set_ylim(-residual_zoom_limit, residual_zoom_limit)


# Display the plots
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

# --- Model Improvement: Using a Random Forest Regressor ---

print("--- Training a more powerful Random Forest model... ---")

# Initialize the Random Forest model.
# n_estimators=100 means we are building 100 decision trees.
# random_state=42 ensures we get the same result every time.
# n_jobs=-1 uses all available CPU cores to speed up training.
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# Train the model on the same training data as before
rf_model.fit(X_train_new, y_train_new)

# Make predictions on the test data with our new model
y_pred_rf = rf_model.predict(X_test_new)

# --- Evaluate the New Random Forest Model ---
mse_rf = mean_squared_error(y_test_new, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print("\n--- Model Performance Comparison ---")
# The previous RMSE was ~10.55 minutes, let's see how we did.
print(f"Previous Decision Tree RMSE: 10.55 minutes")
print(f"New Random Forest RMSE:      {rmse_rf:.2f} minutes")

improvement = 10.55 - rmse_rf
print(f"This is an improvement of {improvement:.2f} minutes in average prediction error.")

# --- Visualize the Improved Predictions ---

# Calculate zoom limits just like before
limit_value_rf = max(np.percentile(y_test_new, 95), np.percentile(y_pred_rf, 95))
zoom_limit_rf = limit_value_rf * 1.1

residuals_rf = y_test_new - y_pred_rf
residual_zoom_limit_rf = np.percentile(np.abs(residuals_rf), 95) * 1.2

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Plot 1: Actual vs. Predicted (Random Forest)
ax1.scatter(y_test_new, y_pred_rf, alpha=0.5, label='Random Forest Predictions')
ax1.plot([0, zoom_limit_rf], [0, zoom_limit_rf], 'r--', lw=2, label='Perfect Prediction')
ax1.set_xlabel('Actual Accident Duration (minutes)', fontsize=12)
ax1.set_ylabel('Predicted Accident Duration (minutes)', fontsize=12)
ax1.set_title('Random Forest: Actual vs. Predicted (Zoomed In)', fontsize=16)
ax1.grid(True)
ax1.set_xlim(0, zoom_limit_rf)
ax1.set_ylim(0, zoom_limit_rf)
ax1.legend()

# Plot 2: Residual Plot (Random Forest)
ax2.scatter(y_pred_rf, residuals_rf, alpha=0.5, edgecolors='g')
ax2.axhline(y=0, color='r', linestyle='--', lw=2)
ax2.set_xlabel('Predicted Accident Duration (minutes)', fontsize=12)
ax2.set_ylabel('Residual (Error) in minutes', fontsize=12)
ax2.set_title('Residual Plot (Random Forest)', fontsize=16)
ax2.grid(True)
ax2.set_xlim(0, zoom_limit_rf)
ax2.set_ylim(-residual_zoom_limit_rf, residual_zoom_limit_rf)

plt.tight_layout()
plt.show()
